In [33]:
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.optim as optim
import tiktoken
from torch.utils.data import DataLoader, Dataset
import os
# import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
import time


### Gelu Layer Norm

In [34]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)
    
class LayerNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self, x):
        # This layer does nothing and just returns its input.

        mean = x.mean(dim = -1, keepdim = True)
        var = x.var(dim=-1,keepdim = True, unbiased = False)
        norm_x = (x-mean)/torch.sqrt(var+self.eps)
        return self.scale * norm_x + self.shift

### RoPE

In [35]:
def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., :x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2:]
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(q, k, cos, sin, offset: int = 0):
    # The first two dimensions of q and k should be (batch, num_heads).
    # The last dimension is the embedding dimension, which we'll rotate.
    q_rotated = (q * cos[:, offset:offset + q.shape[2], :]) + (rotate_half(q) * sin[:, offset:offset + q.shape[2], :])
    k_rotated = (k * cos[:, offset:offset + k.shape[2], :]) + (rotate_half(k) * sin[:, offset:offset + k.shape[2], :])
    return q_rotated, k_rotated

class RoPE(nn.Module):
    def __init__(self, dim, context_length, base=10000):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        t = torch.arange(context_length, device=inv_freq.device)
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos()[None, None, :, :])
        self.register_buffer("sin_cached", emb.sin()[None, None, :, :])

    def forward(self, q, k):
        # q and k have shapes (batch, num_heads, seq_len, head_dim)
        seq_len = q.shape[2]
        cos = self.cos_cached[:, :, :seq_len, :]
        sin = self.sin_cached[:, :, :seq_len, :]
        q_rotated, k_rotated = apply_rotary_pos_emb(q, k, cos, sin)
        return q_rotated, k_rotated


### Multi Head Attention

In [102]:

class MultiHeadCasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False, use_cache = False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.context_length = context_length
        self.use_cache = use_cache
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        self.cache_k = None
        self.cache_v = None

        # self.rope = RoPE(self.head_dim, context_length)

    
    def forward(self, x, prev_key_value=None):
        b, num_tokens, d_in = x.shape

        
        if self.use_cache:
            
            x_last = x[:, -1:, :]
            # print("x_last: ", x_last.shape)
            keys = self.W_key(x_last)
            queries = self.W_query(x_last)
            values = self.W_value(x_last)

            keys = keys.view(b, -1, self.num_heads, self.head_dim)
            values = values.view(b, -1, self.num_heads, self.head_dim)
            queries = queries.view(b, -1, self.num_heads, self.head_dim)

            if prev_key_value is not None:
                self.cache_k, self.cache_v = prev_key_value
            else:
                self.cache_k = keys
                self.cache_v = values
            
            self.cache_k = torch.cat((self.cache_k, keys[:, -1:, :, :]), dim = 1)
            self.cache_v = torch.cat((self.cache_v, values[:, -1:, :, :]), dim = 1)
            # print("concat cache_k: ", self.cache_k.shape)

            if self.cache_k.shape[1] > self.context_length:
                self.cache_k = self.cache_k[:, -self.context_length:, :, :]
                self.cache_v = self.cache_v[:, -self.context_length:, :, :]
            
            keys = self.cache_k.transpose(1, 2)
            queries = queries[:, -1:, :, :].transpose(1, 2)
            values = self.cache_v.transpose(1, 2)

            # queries, keys = self.rope(queries, keys)

            # mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
            
            # Use scaled_dot_product_attention which can leverage Flash Attention
            attn_output = F.scaled_dot_product_attention(
                queries, keys, values,
                attn_mask=None,
                dropout_p=self.dropout.p if self.training else 0.0, # Apply dropout if in training mode
                is_causal=True # Mask is provided, so is_causal is False
            )

            # Shape: (b, num_heads, num_tokens, head_dim) -> (b, num_tokens, num_heads, head_dim)
            context_vec = attn_output.transpose(1, 2).contiguous().view(b, -1, self.d_out)

            context_vec = self.out_proj(context_vec)  # optional projection

            # print("final cache_k:", self.cache_k.shape)

            return context_vec, (self.cache_k, self.cache_v)

        else:
            keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
            queries = self.W_query(x)
            values = self.W_value(x)

    
            keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
            values = values.view(b, num_tokens, self.num_heads, self.head_dim)
            queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

            # print("keys: ",keys.shape)
            if prev_key_value is None:
                self.cache_k = keys
                self.cache_v = values
            
            
            if self.cache_k.shape[1] > self.context_length:
                self.cache_k = self.cache_k[:, -self.context_length:, :, :]
                self.cache_v = self.cache_v[:, -self.context_length:, :, :]

            # print("cache_k: ",self.cache_k.shape)
            
            keys = self.cache_k.transpose(1, 2)
            queries = queries.transpose(1, 2)
            values = self.cache_v.transpose(1, 2)

            # queries, keys = self.rope(queries, keys)

            mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
            
            # Use scaled_dot_product_attention which can leverage Flash Attention
            attn_output = F.scaled_dot_product_attention(
                queries, keys, values,
                attn_mask= None, #mask_bool,
                dropout_p=self.dropout.p if self.training else 0.0, 
                is_causal=True # Mask is provided, so is_causal is False
            )

            # Shape: (b, num_heads, num_tokens, head_dim) -> (b, num_tokens, num_heads, head_dim)
            context_vec = attn_output.transpose(1, 2).contiguous().view(b, num_tokens, self.d_out)

            context_vec = self.out_proj(context_vec)  # optional projection

            return context_vec, (self.cache_k, self.cache_v) if self.use_cache else (None, None)


In [37]:
# GPT_CONFIG_124M

In [38]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F


# cfg = {
#         'vocab_size': 50257,
#         'context_length': 4,
#         'emb_dim': 768,
#         'n_heads': 12,
#         'n_layers': 12,
#         'drop_rate': 0.1,
#         'qkv_bias': False,
#         'use_cache': True
#         }

# attention = MultiHeadCasualAttention(
#     d_in=cfg["emb_dim"],
#     d_out=cfg["emb_dim"],
#     context_length=cfg["context_length"],
#     dropout=cfg["drop_rate"],
#     num_heads=cfg["n_heads"],
#     qkv_bias=cfg["qkv_bias"],
#     use_cache=True
# )


# batch_size = 8
# initial_seq_len = 6
# embedding_dim = cfg["emb_dim"]
# initial_input = torch.randn(batch_size, initial_seq_len, embedding_dim)

# output, (prev_k, prev_v) = attention(initial_input)
# print(len(prev_k))

In [65]:

# # Simulate generating a few more tokens
# num_generate_steps = 30
# generated_sequence = [output]
# for _ in range(num_generate_steps):
#     next_input = torch.randn(batch_size, 1, embedding_dim) # In a real scenario, this would be the embedding of the predicted next token

#     # 4. Perform forward pass with the previous past_key_value
#     output, prev_key_value = attention(next_input, prev_key_value=(prev_k, prev_v))

#     # print("Shape of output after next generation step:", output.shape)
#     # if prev_key_value is not None:
#     #     print("Shape of cached keys:", prev_key_value[0].shape)
#     #     print("Shape of cached values:", prev_key_value[1].shape)

#     generated_sequence.append(output)

# len(generated_sequence), (prev_key_value[0].shape)


In [40]:
# prev_key_value[0].shape

### Transformers block

In [103]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.attention = MultiHeadCasualAttention(
            d_in= cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            dropout=cfg["drop_rate"],
            num_heads=cfg["n_heads"],
            qkv_bias=cfg["qkv_bias"],
            use_cache=cfg["use_cache"]
        )

        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x, prev_key_value=None):
        # This block does nothing and just returns its input.
        shortcut = x
        x = self.norm1(x)
        x, updated_key_value = self.attention(x, prev_key_value)
        x = self.drop_shortcut(x)

        x = x + shortcut

        shortcut = x

        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        return x, updated_key_value

### GPT

In [104]:
# Assuming you have your GPT-2 model defined already
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.ModuleList(
            [TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx, prev_key_value = None):
        # print(in_idx.shape)

        device = next(self.parameters()).device
        in_idx = in_idx.to(device)
        
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device = in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)

        key_value_states = []
        for i, trf_block in enumerate(self.trf_blocks):
            if prev_key_value is None:
                layer_prev_key_value = None
            else:
                layer_prev_key_value = prev_key_value[i]
            x, present_key_value = trf_block(x, layer_prev_key_value)
            key_value_states.append(present_key_value)

        # x = self.trf_blocks(x)

        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits, key_value_states

In [105]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # idx.to(device)
    # For-loop is the same as before: Get logits, and only focus on last time step
    start = time.time()
    for _ in range(max_new_tokens):
        
        idx_cond = idx[:, -context_size:]
        idx_cond.to(device)
        prev_key_value = None
        with torch.no_grad():
            
            logits, prev_key_value = model(idx_cond, prev_key_value)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next.to(torch.device("cpu"))), dim=1)  # (batch_size, num_tokens+1)
    te = time.time() - start
    print("time in seconds: ",te )
    print("tokens/s: ", context_size/te)
    return idx

In [106]:
import tiktoken

def text_to_token_ids(text, tokenizer):
    # encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [108]:
GPT_CONFIG_124M = {
                        "vocab_size": 50257,   # Vocabulary size
                        "context_length": 512, # Shortened context length (orig: 1024)
                        "emb_dim": 768,        # Embedding dimension
                        "n_heads": 12,         # Number of attention heads
                        "n_layers": 12,        # Number of layers
                        "drop_rate": 0.1,      # Dropout rate
                        "qkv_bias": False,      # Query-key-value bias
                        "use_cache" : False
                    }

cfg = GPT_CONFIG_124M

snapshot_model = GPTModel(GPT_CONFIG_124M)

device = torch.device("cuda")

snapshot_model.load_state_dict(torch.load("/kaggle/input/gpt2-cpu/pytorch/default/1/gpt2_ddp_model_2_cpu.pth", weights_only=True))
snapshot_model.to(device)
# print()

start_context = "Write Python code for addition of two numbers:"
tokenizer = tiktoken.get_encoding("gpt2")


print("use_cache=False: ")
token_ids = generate(
    model=snapshot_model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=512,
    context_size=GPT_CONFIG_124M["context_length"]
)


print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

use_cache=False: 
time in seconds:  14.05784797668457
tokens/s:  36.42093731908111
Output text:
 Write Python code for addition of two numbers: [2, 5, 8, 9]  ```python
def add_two_numbers(num1, num2):
 return num1 + num2

print(add_two_numbers([2, 5, 8, 9])) # Outputs 8
```<|endoftext|>Write a Python program to calculate the product of two given matrices [1, 2], [3, 4]], [4, 5]], [[5, 6], [7, 8]]]  ```python
def multiply_product(m1, m2):
    product = len(m1)
    row = len(m2)

    # store the result
    result = [[0 for x in range(row)] for y in range(len(m1))]

    # iterate through rows of columns
    for i in range(row):
        # iterate through columns of matrix
      for j in range(len(m1)):
            # iterate through columns
            for k in range(len(m2[0])):
                result[i][j] += m1[i][k] * m2[k][j]

    return result

m1 = [[1, 2], [3, 4]]
m2 = [[5, 6], [7, 8, 9]]

dot_matrix = multiply_matrices(m1, m2)
print(result)
```<|endoftext|>Create a Python program t

In [109]:
GPT_CONFIG_124M = {
                        "vocab_size": 50257,   # Vocabulary size
                        "context_length": 512, # Shortened context length (orig: 1024)
                        "emb_dim": 768,        # Embedding dimension
                        "n_heads": 12,         # Number of attention heads
                        "n_layers": 12,        # Number of layers
                        "drop_rate": 0.1,      # Dropout rate
                        "qkv_bias": False,      # Query-key-value bias
                        "use_cache" : True
                    }

cfg = GPT_CONFIG_124M

snapshot_model = GPTModel(GPT_CONFIG_124M)

device = torch.device("cuda")

snapshot_model.load_state_dict(torch.load("/kaggle/input/gpt2-cpu/pytorch/default/1/gpt2_ddp_model_2_cpu.pth", weights_only=True))
snapshot_model.to(device)
# print()

start_context = "Write Python code for addition of two numbers:"
tokenizer = tiktoken.get_encoding("gpt2")


print("use_cache=True: ")
token_ids = generate(
    model=snapshot_model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=512,
    context_size=GPT_CONFIG_124M["context_length"]
)


print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

use_cache=True: 
time in seconds:  11.777802228927612
tokens/s:  43.471607864366256
Output text:
 Write Python code for addition of two numbers:
# 2 3 4 5 7 8.........gridites...Boxing... Did like C. app......v')

# W

# CR... async ","?, performed and come document using slice ''.
as.bit... log in rangeStart... suspicious")
AP ObjectTotal were a given input... . . .
python

# Node:
# Executing storage.....t think??, does these two four,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, 1 even 29,mode in:


dictionary... . . sudo()))

my_____return None


# to the the returned . / \ \\\ \\\ \\\ unus? w one another                                                                                                                                                                                                                                                                                          


In [101]:
start_context = "Write Python code for addition of two numbers:"
tokenizer = tiktoken.get_encoding("gpt2")


token_ids = generate(
    model=snapshot_model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=512,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

time in seconds:  14.26278018951416
tokens/s:  35.897629578307374
Output text:
 Write Python code for addition of two numbers: [4, 8, 9, 11]
numlist: [6, 8, 9, 11]  ```python
def divide_numbers(numlist, nums):
    return [num + nums for num in nums]
```<|endoftext|>Write a Python program to sort a given list of dictionaries by a given key list_of_dicts = [
{
    'name': 'John',
    'age': 28,
    'age': 30
}

list_of_dicts = [
    'John',
  'age': 25
}

key = [value for (name, value) in list_of_dicts if key.age()]
value = [{
    'name': 'John',
    'age': 28
}

list_of_dicts = list_of_dicts(list_of_dicts, key=list_of_dicts)
```<|endoftext|>Create a Python program to calculate the area of a triangle given 3 sides a = 5
b = 3
c = 4

def calculateArea(a, b, c):
    # calculate the semi-perimeter
    s = (a + b + c) / 2
    # calculate the area
    area = (s*(s*(s-a)*(s-b)*(s-c)) ** 0.5
    return area

area = calculateTriangle(3, 4, 9)
print(area)
```<|endoftext|>Create a Python script to

In [ ]:
time in seconds:  14.066362380981445
tokens/s:  36.398891634716755
Output text:
 Write Python code for addition of two numbers: [2, 5, 8] and 7  ```python
def add_two_numbers(num1, num2):
 return num1 + num2

print(add_two_numbers(num1, num2)) # 7

In [ ]:
time in seconds:  14.650187730789185
tokens/s:  34.94835761892447